# Westminster Dog Show Data
## Part 3: merging BIS, groups cleaned df with `WKC_merged`

In [12]:
# housekeeping
import pprint as pp
from tqdm import tqdm
from pathlib import Path

# analysis
import pandas as pd
import numpy as np

In [44]:
#dfs import
BIS_path = Path('./data/BIS_data.csv')
BIS_raw = pd.read_csv(BIS_path)
BIS_df = BIS_raw.copy()

groups_path = Path('./data/groups_data.csv')
groups_raw = pd.read_csv(groups_path)
groups_df = groups_raw.copy()

In [61]:
# val clean up
groups_df['year'] = groups_df['year'].astype(int)
groups_df['year'].value_counts()

#locating index where year = 1
groups_df['year'].min()
groups_df[groups_df['year'] == 1]
idx_1 = 453
#rewrite
groups_df['year'].iloc[idx_1] = 1964
groups_df.iloc[idx_1]

#locating index where year = 202
groups_df['year'].min()
groups_df[groups_df['year'] == 202]
idx_202 = 294
#rewrite
groups_df['year'].iloc[idx_202] = 2023
groups_df.iloc[idx_202]

/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_38231/3413539797.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_df['year'].iloc[idx_1] = 1964
/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_38231/3413539797.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_df['year'].iloc[idx_202] = 2023


year                                                     2023
breed                          American Staffordshire Terrier
dog               GCHP CH Lbk's Rebel And Proud Party Crasher
owner(s)    Traci Chlan Luisma & Lacey Benitez & Ion Ispas...
group                                                 Terrier
Name: 294, dtype: object

In [60]:
groups_df.head()

,year,breed,dog,owner(s),group
0,2023,English Setter,GCHB CH Winchester's An Apple A Day JH,S Nordstrom & C Hodges & R Barnes & S McGraw &...,Sporting
1,2022,English Setter,GCHS CH Ciara N' Honeygait Belle Of The Ball F...,Van Jacobsen & Lee Afdahl & Amanda & Vito Ciar...,Sporting
2,2021,German Shorthaired Pointer,GCHS CH Clarity Reach The sky VJK-Myst,V. Nunes-Atkinson & Yvonne Hassler-Deterding &...,Sporting
3,2020,Golden Retriever,GCHP CH Hillock's Jack Daniel's RA JH CA RATN ...,"Tammy Tomlinson , Jim Cohen & Robert Samios",Sporting
4,2019,Sussex Spaniel,GCh. Kamand’s Full Of Beans @ Erinhill,Karen Ann Toner & Amanda W Toner,Sporting
